# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f29d45dc9a0>
├── 'a' --> tensor([[-1.8462,  0.2243,  0.8181],
│                   [-0.8258,  0.8124,  0.4211]])
└── 'x' --> <FastTreeValue 0x7f29d45dca60>
    └── 'c' --> tensor([[ 0.2484,  0.1564, -0.5340,  0.9071],
                        [ 0.7290,  2.7745, -0.0589,  1.0766],
                        [ 0.3423,  0.4663, -1.1750, -0.2259]])

In [4]:
t.a

tensor([[-1.8462,  0.2243,  0.8181],
        [-0.8258,  0.8124,  0.4211]])

In [5]:
%timeit t.a

89.8 ns ± 1.08 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f29d45dc9a0>
├── 'a' --> tensor([[-0.8320,  0.1169,  2.0937],
│                   [-1.4211, -0.5322, -0.7525]])
└── 'x' --> <FastTreeValue 0x7f29d45dca60>
    └── 'c' --> tensor([[ 0.2484,  0.1564, -0.5340,  0.9071],
                        [ 0.7290,  2.7745, -0.0589,  1.0766],
                        [ 0.3423,  0.4663, -1.1750, -0.2259]])

In [7]:
%timeit t.a = new_value

88.8 ns ± 0.311 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.8462,  0.2243,  0.8181],
               [-0.8258,  0.8124,  0.4211]]),
    x: Batch(
           c: tensor([[ 0.2484,  0.1564, -0.5340,  0.9071],
                      [ 0.7290,  2.7745, -0.0589,  1.0766],
                      [ 0.3423,  0.4663, -1.1750, -0.2259]]),
       ),
)

In [10]:
b.a

tensor([[-1.8462,  0.2243,  0.8181],
        [-0.8258,  0.8124,  0.4211]])

In [11]:
%timeit b.a

78.5 ns ± 0.74 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2754, -0.7584, -1.6150],
               [-0.2679, -1.0617, -0.3047]]),
    x: Batch(
           c: tensor([[ 0.2484,  0.1564, -0.5340,  0.9071],
                      [ 0.7290,  2.7745, -0.0589,  1.0766],
                      [ 0.3423,  0.4663, -1.1750, -0.2259]]),
       ),
)

In [13]:
%timeit b.a = new_value

644 ns ± 4.46 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.08 µs ± 6.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 79.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

176 µs ± 1.68 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

182 µs ± 7.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f28f6668130>
├── 'a' --> tensor([[[-1.8462,  0.2243,  0.8181],
│                    [-0.8258,  0.8124,  0.4211]],
│           
│                   [[-1.8462,  0.2243,  0.8181],
│                    [-0.8258,  0.8124,  0.4211]],
│           
│                   [[-1.8462,  0.2243,  0.8181],
│                    [-0.8258,  0.8124,  0.4211]],
│           
│                   [[-1.8462,  0.2243,  0.8181],
│                    [-0.8258,  0.8124,  0.4211]],
│           
│                   [[-1.8462,  0.2243,  0.8181],
│                    [-0.8258,  0.8124,  0.4211]],
│           
│                   [[-1.8462,  0.2243,  0.8181],
│                    [-0.8258,  0.8124,  0.4211]],
│           
│                   [[-1.8462,  0.2243,  0.8181],
│                    [-0.8258,  0.8124,  0.4211]],
│           
│                   [[-1.8462,  0.2243,  0.8181],
│                    [-0.8258,  0.8124,  0.4211]]])
└── 'x' --> <FastTreeValue 0x7f28fea5c580>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.8 µs ± 293 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f28f6668610>
├── 'a' --> tensor([[-1.8462,  0.2243,  0.8181],
│                   [-0.8258,  0.8124,  0.4211],
│                   [-1.8462,  0.2243,  0.8181],
│                   [-0.8258,  0.8124,  0.4211],
│                   [-1.8462,  0.2243,  0.8181],
│                   [-0.8258,  0.8124,  0.4211],
│                   [-1.8462,  0.2243,  0.8181],
│                   [-0.8258,  0.8124,  0.4211],
│                   [-1.8462,  0.2243,  0.8181],
│                   [-0.8258,  0.8124,  0.4211],
│                   [-1.8462,  0.2243,  0.8181],
│                   [-0.8258,  0.8124,  0.4211],
│                   [-1.8462,  0.2243,  0.8181],
│                   [-0.8258,  0.8124,  0.4211],
│                   [-1.8462,  0.2243,  0.8181],
│                   [-0.8258,  0.8124,  0.4211]])
└── 'x' --> <FastTreeValue 0x7f29d4598070>
    └── 'c' --> tensor([[ 0.2484,  0.1564, -0.5340,  0.9071],
                        [ 0.7290,  2.7745, -0.0589,  1.0766],
                 

In [23]:
%timeit t_cat(trees)

38.1 µs ± 57.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

74.4 µs ± 495 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.8462,  0.2243,  0.8181],
                [-0.8258,  0.8124,  0.4211]],
       
               [[-1.8462,  0.2243,  0.8181],
                [-0.8258,  0.8124,  0.4211]],
       
               [[-1.8462,  0.2243,  0.8181],
                [-0.8258,  0.8124,  0.4211]],
       
               [[-1.8462,  0.2243,  0.8181],
                [-0.8258,  0.8124,  0.4211]],
       
               [[-1.8462,  0.2243,  0.8181],
                [-0.8258,  0.8124,  0.4211]],
       
               [[-1.8462,  0.2243,  0.8181],
                [-0.8258,  0.8124,  0.4211]],
       
               [[-1.8462,  0.2243,  0.8181],
                [-0.8258,  0.8124,  0.4211]],
       
               [[-1.8462,  0.2243,  0.8181],
                [-0.8258,  0.8124,  0.4211]]]),
    x: Batch(
           c: tensor([[[ 0.2484,  0.1564, -0.5340,  0.9071],
                       [ 0.7290,  2.7745, -0.0589,  1.0766],
                       [ 0.3423,  0.4663, -1.1750, -0.2259]],
         

In [26]:
%timeit Batch.stack(batches)

98 µs ± 548 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.8462,  0.2243,  0.8181],
               [-0.8258,  0.8124,  0.4211],
               [-1.8462,  0.2243,  0.8181],
               [-0.8258,  0.8124,  0.4211],
               [-1.8462,  0.2243,  0.8181],
               [-0.8258,  0.8124,  0.4211],
               [-1.8462,  0.2243,  0.8181],
               [-0.8258,  0.8124,  0.4211],
               [-1.8462,  0.2243,  0.8181],
               [-0.8258,  0.8124,  0.4211],
               [-1.8462,  0.2243,  0.8181],
               [-0.8258,  0.8124,  0.4211],
               [-1.8462,  0.2243,  0.8181],
               [-0.8258,  0.8124,  0.4211],
               [-1.8462,  0.2243,  0.8181],
               [-0.8258,  0.8124,  0.4211]]),
    x: Batch(
           c: tensor([[ 0.2484,  0.1564, -0.5340,  0.9071],
                      [ 0.7290,  2.7745, -0.0589,  1.0766],
                      [ 0.3423,  0.4663, -1.1750, -0.2259],
                      [ 0.2484,  0.1564, -0.5340,  0.9071],
                      [ 0.7290,  

In [28]:
%timeit Batch.cat(batches)

181 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

452 µs ± 4.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
